# Best bus routes

Find the best bus routes from PTV data.


Many bus routes don't have night services. So I'd like to find the bus routes with the latest final arrival time in any date.

In [1]:
import components as comp
import pandas as pd
import numpy as np

In [2]:
# Utilize a custom module to convert PTV's gtfs.zip data into a Series of Pandas DataFrames

df = comp.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
# 2m30s

In [3]:
ptv = df.set_index(['branch_id', 'table_name'], inplace=False)['df']

In [4]:
# Create a DataFrame for each bus table in the GTFS data
df_bus_agency : pd.DataFrame = ptv['4']['agency']
df_bus_calendar : pd.DataFrame = ptv['4']['calendar']
df_bus_calendar_dates : pd.DataFrame = ptv['4']['calendar_dates']
df_bus_routes : pd.DataFrame = ptv['4']['routes']
df_bus_shapes : pd.DataFrame = ptv['4']['shapes']
df_bus_stops : pd.DataFrame = ptv['4']['stops']
df_bus_stop_times : pd.DataFrame = ptv['4']['stop_times']
df_bus_trips : pd.DataFrame = ptv['4']['trips']

In [5]:
def get_max_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmax()]

def get_min_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmin()]

In [6]:
df_bus_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,43-477--1-MF1-8728614,05:56:00,05:56:00,6725,1,NaN,0,0,0.00
1,43-477--1-MF1-8728614,05:56:00,05:56:00,6726,2,NaN,0,0,289.29
2,43-477--1-MF1-8728614,06:00:00,06:00:00,9095,3,NaN,0,0,1442.47
3,43-477--1-MF1-8728614,06:01:00,06:01:00,27586,4,NaN,0,0,4975.39
4,43-477--1-MF1-8728614,06:02:00,06:02:00,27587,5,NaN,0,0,5339.19


In [7]:
# Find the row with the maximum arrival time for each trip 
dfb1 = df_bus_stop_times.groupby('trip_id')['arrival_time'].max().to_frame(name="arrival_time").reset_index()
# 20s

In [8]:
dfb2 : pd.DataFrame = pd.merge(df_bus_trips, dfb1, on='trip_id', how='left')

In [9]:
dfb3 : pd.DataFrame = dfb2.groupby(['route_id', 'service_id'])['arrival_time'].max().to_frame(name="arrival_time").reset_index()

In [10]:
dfb4 = pd.merge(dfb3, df_bus_calendar, on='service_id', how='left')

In [11]:
# Unpivot the days of the week into a single column
dfb5 = pd.melt(
    frame=dfb4, 
    id_vars=['route_id', 'arrival_time'], 
    value_vars=['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'],
    var_name='day_of_week', 
    value_name='service_operating'
)
dfb5 = dfb5[dfb5['service_operating'] == 1][['route_id', 'arrival_time', 'day_of_week']]

KeyError: "The following 'id_vars' are not present in the DataFrame: ['final_arrival_time']"

In [ ]:
dfb6 = dfb5.groupby(['route_id', 'day_of_week'])['arrival_time'].max().to_frame(name='arrival_time').reset_index()

In [ ]:
dfb7 = pd.pivot(dfb6, index='route_id', columns='day_of_week', values='arrival_time')
dfb7.reset_index(inplace=True)

In [ ]:
# Reorder columns
dfb7 = dfb7[['route_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']]

In [ ]:
# Remove routes that don't operate on at least 1 particular day
dfb7.dropna(axis=0, how='any', inplace=True, subset=['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])

In [ ]:
dfb7['earliest_final'] = dfb7[['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']].min(axis=1)

In [ ]:
dfb8 = pd.merge(dfb7, df_bus_routes, on='route_id', how='left')

In [ ]:
dfb8.sort_values(by='earliest_final', ascending=False, inplace=True)

In [ ]:
dfb8.head(20)

In [ ]:
dfb_stop_times = pd.merge(df_bus_stop_times, df_bus_trips, on='trip_id', how='left')